In [64]:
from pathlib import Path
from pandas import read_csv
import numpy as np

data_1hour = read_csv(Path("../Data/electricitybalancenonv_2_1_2022.csv"), sep=',', decimal='.', encoding='utf-8', index_col='HourUTC', parse_dates=True)
#data_1hour = data_1hour.drop('HourDK', axis=1)
data_dk1 = data_1hour[data_1hour['PriceArea'] == 'DK1']
data_dk2 = data_1hour[data_1hour['PriceArea'] == 'DK2']

In [53]:
data_dk1

,HourDK,PriceArea,TotalLoad,Biomass,FossilGas,FossilHardCoal,FossilOil,HydroPower,OtherRenewable,SolarPower,Waste,OnshoreWindPower,OffshoreWindPower,ExchangeContinent,ExchangeGreatBelt,ExchangeNordicCountries
HourUTC,,,,,,,,,,,,,,,,
2021-12-31 23:00:00+00:00,2022-01-01T00:00:00,DK1,2028.55,210.24,141.31,177.46,11.44,1.0,2.21,0.72,95.29,948.25,1054.94,-137.0,394.0,-703.0
2021-12-31 22:00:00+00:00,2021-12-31T23:00:00,DK1,2107.70,361.05,179.47,176.25,11.50,1.0,2.18,0.72,102.80,824.58,1082.46,-548.0,593.0,-503.0
2021-12-31 21:00:00+00:00,2021-12-31T22:00:00,DK1,2154.02,228.84,154.46,171.63,10.86,1.0,2.10,0.72,99.39,624.31,992.84,-361.0,593.0,-154.0
2021-12-31 20:00:00+00:00,2021-12-31T21:00:00,DK1,2211.26,153.95,146.71,187.90,10.90,1.0,2.14,0.73,99.02,693.77,791.86,50.0,597.0,-268.0
2021-12-31 19:00:00+00:00,2021-12-31T20:00:00,DK1,2301.86,206.49,154.31,224.50,11.32,1.0,2.20,0.73,99.80,859.29,795.78,-177.0,597.0,-225.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-01 04:00:00+00:00,2020-01-01T05:00:00,DK1,1781.79,172.77,222.83,369.13,10.87,2.0,2.78,0.00,117.44,1852.97,887.99,-1141.0,-71.0,-485.0
2020-01-01 03:00:00+00:00,2020-01-01T04:00:00,DK1,1745.69,163.37,220.16,321.75,10.73,2.0,2.76,0.00,106.83,1732.34,898.73,-1118.0,158.0,-41.0
2020-01-01 02:00:00+00:00,2020-01-01T03:00:00,DK1,1782.09,157.09,219.65,329.49,10.76,2.0,2.77,0.00,107.20,1568.97,829.06,-1393.0,394.0,295.0


In [65]:
data_dk1 = data_dk1.asfreq('1H')
data_dk1[data_dk1.TotalLoad.isna()]

,HourDK,PriceArea,TotalLoad,Biomass,FossilGas,FossilHardCoal,FossilOil,HydroPower,OtherRenewable,SolarPower,Waste,OnshoreWindPower,OffshoreWindPower,ExchangeContinent,ExchangeGreatBelt,ExchangeNordicCountries
HourUTC,,,,,,,,,,,,,,,,
2020-07-25 20:00:00+00:00,2020-07-25T22:00:00,DK1,NaN,3.03,80.00,356.05,14.09,0.71,2.66,27.45,80.16,614.73,527.31,-1449.0,6.0,1662.0
2020-07-25 21:00:00+00:00,2020-07-25T23:00:00,DK1,NaN,2.78,72.11,405.74,12.90,0.72,2.81,8.85,80.03,590.53,528.23,-1111.0,-455.0,1661.0
2020-07-25 22:00:00+00:00,2020-07-26T00:00:00,DK1,NaN,2.79,67.59,404.60,13.11,0.74,3.49,0.00,79.77,600.64,452.32,-1452.0,-180.0,1662.0
2020-07-25 23:00:00+00:00,2020-07-26T01:00:00,DK1,NaN,10.21,103.80,404.10,13.40,0.71,3.34,0.00,79.55,632.98,526.38,-1770.0,-120.0,1662.0
2020-07-26 00:00:00+00:00,2020-07-26T02:00:00,DK1,NaN,11.06,70.96,322.83,13.31,0.71,3.31,0.00,79.94,678.75,712.41,-1955.0,-30.0,1662.0
2020-07-26 01:00:00+00:00,2020-07-26T03:00:00,DK1,NaN,6.26,68.65,340.90,13.15,0.71,3.46,0.00,81.02,658.60,776.33,-1948.0,-101.0,1662.0
2020-07-26 02:00:00+00:00,2020-07-26T04:00:00,DK1,NaN,6.87,68.75,385.54,13.16,0.72,3.47,0.00,80.61,748.11,653.01,-1896.0,-153.0,1661.0
2020-07-26 03:00:00+00:00,2020-07-26T05:00:00,DK1,NaN,7.35,66.96,254.83,13.27,0.71,2.91,0.00,80.13,936.81,796.20,-1990.0,-243.0,1661.0
2020-07-26 05:00:00+00:00,2020-07-26T07:00:00,DK1,NaN,7.45,63.61,380.19,12.90,0.73,2.48,0.00,79.97,1050.34,656.00,-1811.0,-343.0,1661.0


In [60]:
data_dk1['2020-07-25 16:00:00+00:00':'2020-10-26 22:00:00+00:00']

,HourDK,PriceArea,TotalLoad,Biomass,FossilGas,FossilHardCoal,FossilOil,HydroPower,OtherRenewable,SolarPower,Waste,OnshoreWindPower,OffshoreWindPower,ExchangeContinent,ExchangeGreatBelt,ExchangeNordicCountries
HourUTC,,,,,,,,,,,,,,,,
2020-07-25 16:00:00+00:00,2020-07-25T18:00:00,DK1,2233.51,2.91,79.01,331.18,13.47,0.73,2.64,112.89,80.51,501.74,640.46,-1226.0,44.0,1661.0
2020-07-25 17:00:00+00:00,2020-07-25T19:00:00,DK1,2107.80,3.27,84.95,239.09,15.30,0.72,2.83,55.14,80.76,540.85,653.00,-1311.0,85.0,1661.0
2020-07-25 18:00:00+00:00,2020-07-25T20:00:00,DK1,2015.10,3.27,86.30,256.62,15.29,0.72,2.80,27.45,81.56,576.30,638.98,-1338.0,7.0,1661.0
2020-07-25 19:00:00+00:00,2020-07-25T21:00:00,DK1,1935.26,3.24,86.39,400.48,15.13,0.72,2.65,27.45,81.87,588.80,569.08,-1455.0,-7.0,1661.0
2020-07-25 20:00:00+00:00,2020-07-25T22:00:00,DK1,NaN,3.03,80.00,356.05,14.09,0.71,2.66,27.45,80.16,614.73,527.31,-1449.0,6.0,1662.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-26 18:00:00+00:00,2020-10-26T19:00:00,DK1,2878.88,143.01,276.26,552.93,16.34,1.39,3.74,0.00,62.38,1023.92,877.29,-1648.0,-336.0,1908.0
2020-10-26 19:00:00+00:00,2020-10-26T20:00:00,DK1,2687.09,148.84,157.55,267.94,14.52,1.39,3.48,0.00,62.16,1095.66,966.10,-1526.0,-322.0,1829.0
2020-10-26 20:00:00+00:00,2020-10-26T21:00:00,DK1,2514.18,125.61,119.92,219.11,14.73,1.39,3.51,0.00,58.94,1187.17,964.44,-1584.0,-199.0,1617.0


In [66]:
np.mean(data_dk1.loc['2020-07-22 22:00:00+00:00','TotalLoad'], data_dk1.loc['2020-07-22 22:00:00+00:00','TotalLoad'])

1755.87

AttributeError: 'str' object has no attribute 'hour'

In [ ]:
data_dk2

In [ ]:
X, y = data_dk1.drop(columns=['TotalLoad']), data_dk1.TotalLoad